In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_hub = pd.read_excel('../raw_data/hubspot-crm-exports-alle-deals-2022-02-16.xlsx')

/home/pawel/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
df_hub.sample(10)

,Deal ID,Datum Ident,Grund für gewonnenen Abschluss,Jährlicher Vertragswert,Notizen,Datum der letzten Änderung,Berater: Firma (Firmenname),Adresszeile,Ansprache versendet,Berater: Ansprechpartner,...,Aktiv / Passiv akquiriert,Interesse über,Total: MP,LOST | MP,PLZ,Betrag in der Unternehmenswährung,Associated Company ID,Associated Company,Associated Contact IDs,Associated Contacts
18958,6087277497,NaT,NaN,NaN,NaN,2022-02-01 11:22:00,NaN,Karla Engliše 519/11,2021-09-01,https://app.hubspot.com/contacts/4804905/conta...,...,NaN,NaN,NaN,NaT,150 00,NaN,4635398572,AAKON Capital,735930,Jan Matoušek (info@aakoncapital.com)
60260,2293218107,NaT,NaN,NaN,NaN,2021-09-06 03:59:00,NaN,Kaistraße 6,2020-03-05,https://app.hubspot.com/contacts/4804905/conta...,...,NaN,NaN,NaN,NaT,40211,NaN,2309552130,KMP Holding GmbH (Pfenning Gruppe),416351,Karl-Martin Pfenning (martin.pfenning@pfenning...
51017,3257901092,NaT,NaN,NaN,NaN,2022-02-01 11:23:00,NaN,Erzbergerstr.18,2020-10-28,https://app.hubspot.com/contacts/4804905/conta...,...,NaN,NaN,NaN,NaT,68165,NaN,7364028080,BRF,751739,Pedro Faria (pedro.faria@noemail.de)
80143,1820678157,NaT,NaN,NaN,NaN,2021-12-13 10:19:00,NaN,Süderstraße 77,2020-03-04,https://app.hubspot.com/contacts/4804905/conta...,...,NaN,NaN,NaN,NaT,20097,NaN,1237806334,Paragon Partners GmbH,144801,Nico Kranzfelder (nk@paragon.de)
81083,1811744427,NaT,NaN,NaN,NaN,2022-02-01 11:22:00,NaN,Ludwig-Rinn-Straße 14-16,2019-10-10,https://app.hubspot.com/contacts/4804905/conta...,...,NaN,NaN,NaN,NaT,35452,NaN,2484641192,VWR,322951,Bernd Goell (bernd_goell@vwr.com)
65508,2202972048,NaT,NaN,NaN,NaN,2022-02-01 11:23:00,NaN,NaN,2020-05-27,https://app.hubspot.com/contacts/4804905/conta...,...,NaN,NaN,NaN,NaT,94025,NaN,2012555735,Wieland Capital GmbH und Co. KG,163351,Robert Wieland (wieland@wieland-capital.com)
1525,7804352288,NaT,NaN,NaN,"""passt eher weniger zu unserer aktuellen Strat...",2022-02-15 14:30:00,NaN,Gristen 1,2022-02-03,https://app.hubspot.com/contacts/4804905/conta...,...,NaN,NaN,NaN,NaT,9315,NaN,5959981897,Möbel Schubiger AG,1255051,Urs Schubiger (urs.schubiger@schubiger.ch)
66940,2059985012,NaT,NaN,NaN,NaN,2022-02-01 11:23:00,NaN,NaN,2019-05-09,https://app.hubspot.com/contacts/4804905/conta...,...,NaN,NaN,NaN,NaT,NaN,NaN,1935799758,spedition-haeger.de,157301,Christoph Haeger (chaeger@spedition-haeger.de)
36223,4941656143,NaT,NaN,NaN,NaN,2022-02-01 11:23:00,NaN,Spitalwaldstraße 10,2021-04-06,https://app.hubspot.com/contacts/4804905/conta...,...,NaN,NaN,NaN,NaT,91126,NaN,1049556096,Axos MidCap GmbH-Triginta Capital GmbH,2234,Hans-Gereon Früh (frueh@axos-midcap.com)
1604,7803746348,NaT,NaN,NaN,NaN,2022-02-14 10:34:00,NaN,Aiterfeld 4,2022-02-03,https://app.hubspot.com/contacts/4804905/conta...,...,NaN,NaN,NaN,NaT,79677,NaN,1057650939,Eiffage Infra-Bau SE,13851,Axel Langemeyer (axel.langemeyer@eiffage.de)


In [4]:
df_hub.columns

Index(['Deal ID', 'Datum Ident', 'Grund für gewonnenen Abschluss',
       'Jährlicher Vertragswert', 'Notizen', 'Datum der letzten Änderung',
       'Berater: Firma (Firmenname)', 'Adresszeile', 'Ansprache versendet',
       'Berater: Ansprechpartner',
       ...
       'Aktiv / Passiv akquiriert', 'Interesse über', 'Total: MP', 'LOST | MP',
       'PLZ', 'Betrag in der Unternehmenswährung', 'Associated Company ID',
       'Associated Company', 'Associated Contact IDs', 'Associated Contacts'],
      dtype='object', length=150)